---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!pip install transformers==4.3.0
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 15.0 MB/s 
     |████████████████████████████████| 880 kB 61.9 MB/s 
     |████████████████████████████████| 3.3 MB 67.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8855d278653bee601ae32e361942af85780616ac2ea453835d4cfe83c32e5551
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.8 MB/s 
     |████████████████████████████████| 181 kB 69.4 MB/s 
     |████████████████████████████████| 158 kB 79.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 157 kB 67.1 MB/s 
     |████████████████████████████████| 157 kB 72.1 MB/s 
     |████████████████████████████████| 157 kB 66.2 MB/s 
     |████████████████████████████████| 157 kB 83.1 MB/s 
     |████████████████████████████████| 157 kB 63.7 MB/s 
     |████████████████████████████████| 157 kB 78.0 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 156 kB 81.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7d13049feadddfba4c59badf7c3d37fb6f03a0e10fbdb9b481cc35e090fe1753
  Stored in directory: /root/.cache/pip/wheels/3e/31

In [26]:
# pip install transformers-interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 289 kB/s 
     |████████████████████████████████| 793 kB 21.9 MB/s 
     |████████████████████████████████| 1.4 MB 61.3 MB/s 
     |████████████████████████████████| 1.6 MB 53.2 MB/s 
  Attempting uninstall: ipython
    Found existing installation: ipython 7.9.0
    Uninstalling ipython-7.9.0:
      Successfully uninstalled ipython-7.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 7.34.0 which is incompatible.


In [27]:
import wandb

wandb.login()
wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: glad415. Use `wandb login --relogin` to force relogin


In [28]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [29]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [30]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [31]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

bertmodel = AutoModel.from_pretrained("/content/drive/MyDrive/Kobert/TAPT_Model_Save")

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/Kobert/TAPT_Model_Save and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---
# 학습용 데이터셋 전처리
---

In [23]:
data = pd.read_csv('/content/drive/MyDrive/Kobert/emotional_dataset_modifying/final_emotional_Train_dataset_0920.csv')
del data['Unnamed: 0']
data

,content,label
0,접근성 좋아요,1
1,친절함은 좋았으나 4인이 식사를 하는데 밑반찬까지 한그릇씩만 나와서 먹기에 불편 했...,0
2,고기는 맛있는데 서버들 몇명은 불친절하고 주문도 제대로 못받음,0
3,모든 면에서 강추합니다 주변에서 14년 살았는데 젤 좋습니다,1
4,사장님 직원분들 너무 친절하시구 음식도 맛있어요,1
...,...,...
96339,배달빠르고 맛 좋고 항상 정말 최고입니다,1
96340,회가 신선하고 고소했고 다른 사이드 음식들도 맛있었어요 ㅎㅎ 양도 가격대비 푸짐한거...,1
96341,너무 맛있고 같이 보내주신 사이드도 너무 좋았어요 재주문 할꺼같네ㅎㅎ 잘먹었습니다,1
96342,다른거도 주문했고 혼술세트도 마찬가지로 구성이 둘이서 먹을 수 있을 만큼 알차네여 ...,1


In [22]:
# data.to_csv('/content/drive/MyDrive/Kobert/data/final_emotional_Train_dataset_0920.csv')
# valid.to_csv('/content/drive/MyDrive/Kobert/data/final_emotional_Test_dataset_0920.csv')

----

In [33]:
import numpy as np 

total_data_text = list(data['content'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  72.1750602009466
문장길이 최대 :  1919
문장길이 표준편차 :  92.8554070784751
pad_sequences maxlen :  257
전체 문장의 0.9616789836419497%가 maxlen 설정값 이내에 포함됩니다. 


In [34]:
train_data, test_data = train_test_split(data, test_size = 0.1 ,random_state= 632)
print(train_data.shape)
print(test_data.shape)

(86709, 2)
(9635, 2)


---
# 모델
---

In [35]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, bert_tokenizer, max_len):
    
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=True, padding="max_length",
                                     max_length = max_len, return_tensors = 'pt') for i in dataset['content']]
    self.max_len = max_len
    self.label = [np.int64(i) for i in dataset['label']]
  
  def __getitem__(self,i):
    self.input_ids = self.sentences[i]['input_ids']
    self.attention_mask_token = self.sentences[i]['attention_mask']
    self.token_type_ids = self.sentences[i]['token_type_ids']

    return self.input_ids, self.attention_mask_token, self.token_type_ids, self.label[i]

  def __len__(self):
    return len(self.label)

In [36]:
# 하이퍼파라미터 세팅
max_len = 250
batch_size = 16
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 500
learning_rate =  4e-7

In [37]:
data_train = BERTDataset(train_data, tokenizer, max_len)
data_test = BERTDataset(test_data, tokenizer, max_len)

In [38]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)

In [ ]:
# dataloader test 
for a,b,c,d in train_dataloader:
  print(a)
  print(b)
  print(c)
  print(d)
  print('='*100)
  print('input_ids: ',a.shape)
  print('attention_mask: ',b.shape)
  print('token_type_ids: ',c.shape)
  print('label: ',d.shape)
  break


tensor([[[    2,  1176,  2720,  ...,     0,     0,     0]],

        [[    2,  4744,  3611,  ...,     0,     0,     0]],

        [[    2,  1469, 12831,  ...,     0,     0,     0]],

        ...,

        [[    2,  1585,  1595,  ...,     0,     0,     0]],

        [[    2,  5257,  2138,  ...,     0,     0,     0]],

        [[    2,  5124,  2935,  ...,     0,     0,     0]]])
tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])
tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])
tensor([0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
input_ids:  torch.Size([16, 1, 250])
attention_mask:  torch.Size([16, 1, 250])
token_type_i

In [39]:
wandb.init(project = 'kobert_emotional',
                 entity="glad415",
                 config = {
                     "hidden_size" : 768,
                     "max_len" : max_len,
                     "batch_size" : batch_size,
                     "learning_rate" : learning_rate,
                     "epochs" : num_epochs,
                     "optimizer" : 'adamW',
                     "loss" : "CNLoss",
                 })

In [40]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=2,   
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.dense = nn.Linear(hidden_size , num_classes)

        self.dropout = nn.Dropout(p=dr_rate)
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.bert(input_ids = input_ids.long(), token_type_ids = token_type_ids.long(), attention_mask = attention_mask.to(device))
        out = self.dropout(out.pooler_output)

        return self.dense(out)
    
    ############################# Frezee #############################
    # def freeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = False
    
    # def unfreeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = True
    ##################################################################

In [41]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.3).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [42]:
for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        
        optimizer.zero_grad()
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        label = label.long().to(device)

        out = model(input_ids, attention_mask_token, token_type_ids)

        # Loss
        ################################################################################################
        loss = loss_fn(out, label)

        loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # Accuracy
        ################################################################################################
        preds = torch.argmax(out,dim = 1).flatten()
        accuracy = (preds == label).cpu().numpy().mean()

        train_acc += accuracy 
        ################################################################################################

        # F1_Score
        ################################################################################################
        F1_Score = f1_score(preds.cpu().data, label.cpu(), average = 'macro')
        
        total_f1 +=  F1_Score
        ################################################################################################

        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {loss.data.cpu().numpy():.3f}")
            print(f'train acc {train_acc / (batch_id+1):.3f} f1_score:{total_f1 / (batch_id+1):.3f}')
            print('#'*80)

            # WandB
            #############################################################################################################################
            wandb.log({'interval_Loss': loss.data.cpu().numpy(),
                       'interval_Train_acc' : train_acc, 'interval_F1_Score' : total_f1})
            #############################################################################################################################

    print(f"epoch {e+1} epoch_Train_Accuracy {train_acc / (batch_id+1):.3f}, epoch_F1_score {total_f1 / (batch_id+1):.3f}")
    
    # WandB
    #############################################################################################################################
    wandb.log({'epoch_Train_Accuracy' : train_acc / (batch_id+1), 'epoch_F1_score' : total_f1 / (batch_id+1)})
    #############################################################################################################################

    model.eval()
    with torch.no_grad():
      test_f1 = 0.0
      test_acc = 0.0

      for batch_id, (input_ids, attention_mask_token, token_type_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          
          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          label = label.long().to(device)

          out = model(input_ids, attention_mask_token, token_type_ids)

          # Test_Accuracy
          ################################################################################################    
          preds = torch.argmax(out,dim = 1).flatten()
          accuracy = (preds == label).cpu().numpy().mean()

          test_acc += accuracy
          ################################################################################################ 

          # Test_F1_Score
          ################################################################################################
          F1_Score = f1_score(preds.cpu().data, label.cpu(), average = 'macro')
          
          test_f1 +=  F1_Score
          ################################################################################################

      print('-'*40,'Test_Value','-'*40)
      print(f"test_Accuracy {test_acc / (batch_id+1):.3f}, test_F1_score {test_f1/ (batch_id+1):.3f}")
      print('-'*92)
      
      # WandB
      #############################################################################################################################
      wandb.log({'Test_acc' : test_acc / (batch_id+1), 'Test_F1_Score' : test_f1/ (batch_id+1)})
      #############################################################################################################################

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/5419 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.710
train acc 0.438 f1_score:0.417
################################################################################
epoch 1 batch id 501 loss 0.457
train acc 0.684 f1_score:0.663
################################################################################
epoch 1 batch id 1001 loss 0.098
train acc 0.803 f1_score:0.790
################################################################################
epoch 1 batch id 1501 loss 0.119
train acc 0.846 f1_score:0.835
################################################################################
epoch 1 batch id 2001 loss 0.463
train acc 0.867 f1_score:0.858
################################################################################
epoch 1 batch id 2501 loss 0.090
train acc 0.882 f1_score:0.874
################################################################################
epoch 1 batch id 3001 loss 0.013
train acc 0.892 f1_score:0.884
######################################################################

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/602 [00:00<?, ?it/s]

---------------------------------------- Test_Value ----------------------------------------
test_Accuracy 0.947, test_F1_score 0.943
--------------------------------------------------------------------------------------------


In [43]:
wandb.finish()

Test_F1_Score,▁
Test_acc,▁
epoch_F1_score,▁
epoch_Train_Accuracy,▁
interval_F1_Score,▁▂▂▃▄▄▅▆▇▇█
interval_Loss,█▅▂▂▆▂▁▄▁▁▂
interval_Train_acc,▁▂▂▃▄▄▅▆▇▇█
Test_F1_Score,0.94251
Test_acc,0.94684
epoch_F1_score,0.90831
epoch_Train_Accuracy,0.91428


In [82]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Kobert/model/klue_emotional_dict_0920.pt')

In [ ]:
# model = BERTClassifier(bertmodel,dr_rate=0.3).to(device)
# model.load_state_dict(torch.load('/content/drive/MyDrive/Kobert/model/koBERT_emotional_dict_0914.pt', map_location=device))

<All keys matched successfully>

In [44]:
def softmax(x):
  exp_x = np.exp(x)
  result = exp_x / np.sum(exp_x)

  return result

def predict(predict_sentence):

    dataset_another = pd.DataFrame({'content': predict_sentence, 'label' : 0 }, index = [0])

    another_test = BERTDataset(dataset_another,tokenizer, max_len)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)
    
    model.eval()

    for input_ids, attention_mask_token, token_type_ids, label in test_dataloader:

          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          out = model(input_ids, attention_mask_token, token_type_ids)
          negative, positive = softmax(out[0].detach().cpu().numpy())

          print(f'긍정 점수: {positive:.3f}')
          print(f'부정 점수: {negative:.3f}')


          print(softmax(out[0].detach().cpu().numpy()))
          

---
# 결과값 테스트
---

In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
predict('맛이 없다')

긍정 점수: 0.019
부정 점수: 0.981
[0.9805808 0.0194192]


In [47]:
predict('서비스가 좋아서 다시 가고 싶어요')

긍정 점수: 0.993
부정 점수: 0.007
[0.00679043 0.9932096 ]


In [48]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

긍정 점수: 0.026
부정 점수: 0.974
[0.9741424  0.02585767]


In [49]:
predict('음식은 겉바속촉 매장은 깔끔')

긍정 점수: 0.875
부정 점수: 0.125
[0.12500966 0.8749903 ]


In [50]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

긍정 점수: 0.994
부정 점수: 0.006
[0.00602517 0.99397486]


In [51]:
predict('맛 양 배달 모두 만족합니다')

긍정 점수: 0.992
부정 점수: 0.008
[0.00757157 0.9924284 ]


In [52]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

긍정 점수: 0.992
부정 점수: 0.008
[0.00771289 0.9922871 ]


In [53]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

긍정 점수: 0.332
부정 점수: 0.668
[0.6681004 0.3318996]


In [54]:
predict('사업 번창하세요')

긍정 점수: 0.980
부정 점수: 0.020
[0.01995149 0.9800485 ]


In [55]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고 ㅜㅜ')

긍정 점수: 0.011
부정 점수: 0.989
[0.9885583  0.01144172]


In [56]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

긍정 점수: 0.992
부정 점수: 0.008
[0.00796999 0.99203   ]


In [84]:
predict('맛 없음')

긍정 점수: 0.019
부정 점수: 0.981
[0.9808816  0.01911844]


In [58]:
predict('배달은 빠른데 음식이 맛없어')

긍정 점수: 0.011
부정 점수: 0.989
[0.9885443  0.01145573]


In [59]:
predict('음식이 맛없는데 배달만 빠르네')

긍정 점수: 0.010
부정 점수: 0.990
[0.989881   0.01011901]


In [60]:
predict('배달이 빠름')

긍정 점수: 0.165
부정 점수: 0.835
[0.83507705 0.16492295]


In [61]:
predict('배달은 빠름')

긍정 점수: 0.918
부정 점수: 0.082
[0.0822883 0.9177117]


In [62]:
predict('음식이 식어서와서 화가났지만 한입먹고 맛있어서 화가 풀렸어요')

긍정 점수: 0.975
부정 점수: 0.025
[0.02547782 0.9745222 ]


In [63]:
predict('순대에서 느끼하다')

긍정 점수: 0.018
부정 점수: 0.982
[0.9824919 0.0175081]


In [64]:
predict('배달이 느려서 별로에요 음식은 맛있었는데')

긍정 점수: 0.018
부정 점수: 0.982
[0.9822024  0.01779758]


In [65]:
predict('음식은 맛있어요')

긍정 점수: 0.992
부정 점수: 0.008
[0.00846204 0.9915379 ]


In [66]:
predict('배달이 느려서 별로에요')

긍정 점수: 0.017
부정 점수: 0.983
[0.9833913  0.01660872]


---
# inference
---

In [69]:
original = pd.read_csv('/content/drive/MyDrive/Kobert/emotional_dataset_modifying/final_emotional_Test_dataset_0920.csv')
del original['Unnamed: 0']
original

,content,label
0,맛있고 양도 많고 배달빨리오네요 맛감자서비스 잘먹었습니다,1
1,맛없음 국물이 쓰고 잡내남 김치도 별로임,0
2,이딴업체에다가 돈주고 제본하지마세요 100페이지정도 제본뜨러갔다가 1시간기다리라하길...,0
3,배달도 생각보다 금방이었고 다리랑 윙봉만 있는 구성품이라 먹기도 좋고 맛있네요,1
4,아깝다,0
...,...,...
9995,원래 밀면이 쏘쏘인 나도 맛있었음 밀면 맛을 잘 몰라서인지 부산보다 괜찮았음 비빔보...,1
9996,맛있었어요 버터밥도 맛있구 직원분들도 친절했습니다,1
9997,맛잇습니다 막국수가 참 맛잇어서 다먹고 아쉬움에 헛 젓가락질햇네요 원래 간 안되어잇...,1
9998,난 분명히 싱글레귤러 시켰는데 왜 싱글킹같은 싱글레귤러가 나오지 콘도 크고 컵도 꽉...,1


In [71]:
modeling = original.copy()
modeling['label'] = 0
modeling

,content,label
0,맛있고 양도 많고 배달빨리오네요 맛감자서비스 잘먹었습니다,0
1,맛없음 국물이 쓰고 잡내남 김치도 별로임,0
2,이딴업체에다가 돈주고 제본하지마세요 100페이지정도 제본뜨러갔다가 1시간기다리라하길...,0
3,배달도 생각보다 금방이었고 다리랑 윙봉만 있는 구성품이라 먹기도 좋고 맛있네요,0
4,아깝다,0
...,...,...
9995,원래 밀면이 쏘쏘인 나도 맛있었음 밀면 맛을 잘 몰라서인지 부산보다 괜찮았음 비빔보...,0
9996,맛있었어요 버터밥도 맛있구 직원분들도 친절했습니다,0
9997,맛잇습니다 막국수가 참 맛잇어서 다먹고 아쉬움에 헛 젓가락질햇네요 원래 간 안되어잇...,0
9998,난 분명히 싱글레귤러 시켰는데 왜 싱글킹같은 싱글레귤러가 나오지 콘도 크고 컵도 꽉...,0


In [72]:
another_test = BERTDataset(modeling,tokenizer, max_len)
valid_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)

model.eval()

for batch_id, (input_ids, attention_mask_token, token_type_ids, label) in enumerate(tqdm(valid_dataloader)):

      input_ids = input_ids.long().to(device)
      input_ids = input_ids.squeeze(1)

      token_type_ids = token_type_ids.long().to(device)
      token_type_ids = token_type_ids.squeeze(1)
      
      attention_mask_token= attention_mask_token.squeeze(1)

      out = model(input_ids, attention_mask_token, token_type_ids)
      negative, positive = softmax(out[0].detach().cpu().numpy())

      if positive > negative :
        modeling['label'][batch_id] = 1
      else :
        modeling['label'][batch_id] = 0

100%|██████████| 10000/10000 [02:58<00:00, 55.94it/s]


In [81]:
inference =  original.compare(modeling)
error_rate = len(inference) / len(original) * 100
print(f'{error_rate:.2f}')

5.81
